## 移除最后conv最后一层网络

把 resnet最后一层再拼接回去，重新开始训练 ，finetune


In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
import torch.utils.data as data
from torchvision import transforms

from data_loader import get_loader
from data_loader import get_encoder_loader
from model import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [2]:
batch_size = 256
num_epochs = 10
extract_size = 2048
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize([224,224]),
#     transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_vaild = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### load encoded datas

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# exract the images to embedding tensor
# remove the last conv layer
encoder = EncoderClearCut()
encoder = encoder.to(device)

data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size)
data_loader.dataset.save_to("./bottle_neck/resnet152_train_clear_remove_last_conv.pkl")
# data_loader = get_encoder_loader(transform_train, encoder, device, mode='train', batch_size=batch_size, file='./bottle_neck/resnet152_train_clear_remove_last_conv.pkl')

encoding 10974/32739->33.52%, spent_time:9:27.55

RuntimeError: CUDA error: out of memory

### init model

拼接resnet 最后一个 conv layer，用来训练

In [39]:
# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = ConvClassify(extract_size, class_size)
classify_model = classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# with RMSprop to slow the desent gradient progress
optimizer = torch.optim.Adam(classify_model.parameters(), lr=0.001)

### Load the best trained model,yet!

In [8]:
classify_model.load_state_dict(torch.load('./models/class_train_last_conv_layer_last.pkl'))

### time to train model

In [9]:
optimizer = torch.optim.SGD(classify_model.parameters(), lr=0.0001)

In [5]:
classify_model = classify_model.train()
best_acc = 0

for epoch in range(1, num_epochs+1):
    start = time.time()
    for i_step in range(1, total_step+1):
        
        # Ramdomly get samples
        indices = data_loader.dataset.get_train_indices()
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        embeds, targets = next(iter(data_loader))
        
        embeds = embeds.squeeze(1).unsqueeze(2).unsqueeze(2)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(embeds)
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        if i_step%20 == 0:
            # calculate the status
            predict_result = outputs.argmax(1)
            accuracy = torch.sum(predict_result == targets).item() / batch_size * 100
            best_acc = accuracy if accuracy > best_acc else best_acc
            
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.2f%%, Best_acc: %.2f%%\
            ' % (epoch, num_epochs, i_step, total_step, loss.item(), accuracy, best_acc)
            print('\r' + stats, end='')
            sys.stdout.flush()
    print('\n Epoch {}, spent time:{:.2f}s'.format(epoch, time.time()-start))       
    if epoch%50 == 0:
        torch.save(classify_model.state_dict(), os.path.join('./models', 'class_train_last_conv_layer_%d.pkl' % epoch))
torch.save(classify_model.state_dict(), os.path.join('./models', 'class_train_last_conv_layer_last.pkl'))

Epoch [1/100], Step [60/63], Loss: 0.3593, Accuracy: 86.13%, Best_acc: 86.13%            
 Epoch 1, spent time:56.56s
Epoch [2/100], Step [60/63], Loss: 0.2859, Accuracy: 89.45%, Best_acc: 91.41%            
 Epoch 2, spent time:57.24s
Epoch [3/100], Step [60/63], Loss: 0.1689, Accuracy: 94.92%, Best_acc: 94.92%            
 Epoch 3, spent time:57.24s
Epoch [4/100], Step [60/63], Loss: 0.1500, Accuracy: 93.55%, Best_acc: 94.92%            
 Epoch 4, spent time:57.24s
Epoch [5/100], Step [60/63], Loss: 0.0495, Accuracy: 98.05%, Best_acc: 98.44%            
 Epoch 5, spent time:57.25s
Epoch [6/100], Step [60/63], Loss: 0.0778, Accuracy: 97.46%, Best_acc: 98.44%            
 Epoch 6, spent time:57.25s
Epoch [7/100], Step [60/63], Loss: 0.0393, Accuracy: 98.44%, Best_acc: 98.44%            
 Epoch 7, spent time:57.23s
Epoch [8/100], Step [60/63], Loss: 0.0839, Accuracy: 97.27%, Best_acc: 98.44%            
 Epoch 8, spent time:57.24s
Epoch [9/100], Step [60/63], Loss: 0.0581, Accuracy: 98.

KeyboardInterrupt: 

### How good is the model?

In [40]:
# valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size)
# valid_data_loader.dataset.save_to("./bottle_neck/resnet152_valid_rm1layer.h")
valid_data_loader = get_encoder_loader(transform_vaild, encoder, device, mode='valid', batch_size=batch_size, file="./bottle_neck/resnet152_valid_rm1layer.h")

In [41]:
classify_model.load_state_dict(torch.load('./models/class_train_last_conv_layer_last.pkl'))

In [42]:
classify_model = classify_model.eval()
predict = []
for embed, _ in valid_data_loader.dataset:
    embed = embed.unsqueeze(2).unsqueeze(2)
    p = classify_model(embed).argmax().item()
    predict.append(p)
    
df_refer = valid_data_loader.dataset.refer
df_refer["predict"] = predict
df_refer['correct'] = df_refer.predict == df_refer.disease_class
accuracy = (df_refer.correct == True).sum()/len(df_refer)
print('The final accuracy is %.2f%%.' % (accuracy*100))

The final accuracy is 58.47%.


## With 100 epoch the result is the best,yet.

In [23]:
!mv ./models/class_multify_rm1layer_100.pkl ./models/class_multify_rm1layer_good.pkl